# Testbench

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.covariance import EmpiricalCovariance, LedoitWolf
from warnings import warn

import covariance_DRO

sns.set(rc={'figure.figsize':(8,4)})

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/48_Industry_Portfolios.CSV', header=6, nrows=1132)

In [3]:
df

,Unnamed: 0,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,192607,2.37,0.12,-99.99,-5.19,1.29,8.65,2.50,50.21,-0.48,...,7.70,1.94,-23.79,0.07,1.87,4.61,-0.54,2.89,-4.85,5.20
1,192608,2.23,2.68,-99.99,27.03,6.50,16.81,-0.76,42.98,-3.58,...,-2.38,4.88,5.39,-0.75,-0.13,11.83,2.57,5.30,-0.57,6.76
2,192609,-0.57,1.58,-99.99,4.02,1.26,8.33,6.42,-4.91,0.73,...,-5.54,0.06,-7.87,0.25,-0.56,-1.75,0.72,-3.06,-3.14,-3.86
3,192610,-0.46,-3.68,-99.99,-3.31,1.06,-1.40,-5.09,5.37,-4.68,...,-5.08,-2.64,-15.38,-2.20,-4.11,-11.82,-4.28,-5.74,2.07,-8.49
4,192611,6.75,6.26,-99.99,7.29,4.55,0.00,1.82,-6.40,-0.54,...,3.84,1.60,4.67,6.52,4.33,-2.97,3.58,2.21,4.92,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,202006,-2.54,-0.40,-3.27,0.32,-0.72,8.87,2.49,1.30,1.70,...,-0.81,1.40,1.20,5.39,-2.28,-0.31,-1.32,4.34,0.80,-2.30
1128,202007,5.70,6.01,6.97,5.65,7.78,15.59,4.58,4.98,7.68,...,3.54,7.20,5.83,10.47,4.31,0.71,3.85,-1.37,3.01,5.70
1129,202008,-0.38,3.42,5.65,2.65,4.77,7.71,12.22,3.52,4.84,...,7.10,13.04,4.95,8.14,12.31,6.63,2.10,11.81,4.15,10.75
1130,202009,0.25,-3.96,-0.62,0.25,-6.55,10.91,-4.89,-4.39,-0.66,...,3.50,1.34,-2.08,-4.56,-0.78,-5.05,-2.26,-1.94,-4.05,-1.57


In [4]:
df['year'] = (df['Unnamed: 0']/100).astype(int)
df['month'] = (df['Unnamed: 0']%100).astype(int)

In [5]:
train_start = 1990
train_end = 2010
test_end = 2020

In [6]:
train = df.loc[(df.year>=train_start) & (df.year<train_end)]
test = df.loc[(df.year>=train_end) & (df.year<test_end)]
columns = [c for c in train.columns if c not in ['Unnamed: 0', 'year', 'month']]
train_X = train[columns].to_numpy()
test_X = test[columns].to_numpy()

In [7]:
def portfolio_tester(train, test, cov_estimator, verbose=False):
    # estimate covariance matrix and asset allocation
    cov = cov_estimator(train)
    ones = np.ones((cov.shape[0],1))
    w = (cov@ones)/(ones.T@cov@ones)
    # calculate portfolio returns and metrics
    portfolio_returns = test@w
    std = portfolio_returns.std()
    mean = portfolio_returns.mean()
    sharpe = mean/std
    if verbose:
        print('Mean: {}'.format(mean))
        print('Std: {}'.format(std))
        print('Sharpe: {}'.format(sharpe))
    return mean, std, sharpe

# Defining Leave-1-Out Covariance Estimator

In [8]:
def l1o_cv(X, method, epsilon_candidates):
    best_epsilon, best_std = None, float('inf')
    for e in epsilon_candidates:
        returns = []
        for i in range(X.shape[0]):
            # train-validation split
            val = np.expand_dims(train_X[0], 0)
            train = X[np.arange(0, X.shape[0], 1) != i]
            # find the weight
            cov = covariance_DRO.estimate_cov(EmpiricalCovariance(assume_centered=True).fit(train).covariance_, e, method)
            ones = np.ones((cov.shape[0],1))
            w = (cov@ones)/(ones.T@cov@ones)
            # calculate portfolio returns and metrics
            returns.append(val@w)
        # keep track of best epsilon
        if np.std(returns) < best_std:
            best_std = np.std(returns)
            best_epsilon = e
    
    # warn if best epsilon is on border
    if best_epsilon==epsilon_candidates[0]:
        warn('Epsilon on left side of interval')
    if best_epsilon==epsilon_candidates[-1]:
        warn('Epsilon on right side of interval')
    # return estimated covariance matrix
    return covariance_DRO.estimate_cov(EmpiricalCovariance(assume_centered=True).fit(X).covariance_, best_epsilon, method)

def linear_shrink(X, epsilon_candidates):
    best_epsilon, best_std = None, float('inf')
    for e in epsilon_candidates:
        returns = []
        for i in range(X.shape[0]):
            # train-validation split
            val = np.expand_dims(train_X[0], 0)
            train = X[np.arange(0, X.shape[0], 1) != i]
            # find the weight
            emp_cov = EmpiricalCovariance(assume_centered=True).fit(train).covariance_
            mu = np.sum(np.trace(emp_cov)) / emp_cov.shape[0]
            cov = (1. - e) * emp_cov
            cov.flat[::emp_cov.shape[0] + 1] += e * mu
            
            ones = np.ones((cov.shape[0],1))
            w = (cov@ones)/(ones.T@cov@ones)
            # calculate portfolio returns and metrics
            returns.append(val@w)
        # keep track of best epsilon
        if np.std(returns) < best_std:
            best_std = np.std(returns)
            best_epsilon = e
    
    # warn if best epsilon is on border
    if best_epsilon==epsilon_candidates[0]:
        warn('Epsilon on left side of interval')
    if best_epsilon==epsilon_candidates[-1]:
        warn('Epsilon on right side of interval')
    # return estimated covariance matrix
    emp_cov = EmpiricalCovariance(assume_centered=True).fit(train).covariance_
    mu = np.sum(np.trace(emp_cov)) / emp_cov.shape[0]
    cov = (1. - best_epsilon) * emp_cov
    cov.flat[::emp_cov.shape[0] + 1] += best_epsilon * mu
    return cov

# Testing Performance

In [9]:
print('\033[1mEmpirical Covariance:\033[0m')
mean, std, sharpe = portfolio_tester(train_X, test_X, lambda X: EmpiricalCovariance(assume_centered=True).fit(X).covariance_, verbose=True)
print('\n\033[1mLinear Shrikage:\033[0m')
mean, std, sharpe = portfolio_tester(train_X, test_X, lambda X: linear_shrink(X, np.logspace(-5,0,50)), verbose=True)
print('\n\033[1mLedoit-Wolf:\033[0m')
mean, std, sharpe = portfolio_tester(train_X, test_X, lambda X: LedoitWolf(assume_centered=True).fit(X).covariance_, verbose=True)

Empirical Covariance:
Mean: 1.0637898946080815
Std: 4.350982563545323
Sharpe: 0.2444941755273941

Linear Shrikage:
Mean: 1.060513888888889
Std: 4.082322950563175
Sharpe: 0.2597819677011556

Ledoit-Wolf:
Mean: 1.0637833786544566
Std: 4.350436019834737
Sharpe: 0.24452339347237828


In [10]:
print('\033[1mWasserstein:\033[0m')
mean, std, sharpe = portfolio_tester(train_X, test_X, lambda X: l1o_cv(X, 'Wasserstein', np.logspace(-5,4,50)), verbose=True)
print('\n\033[1mKullback-Leibler:\033[0m')
mean, std, sharpe = portfolio_tester(train_X, test_X, lambda X: l1o_cv(X, 'KLdirect', np.logspace(-5,2,50)), verbose=True)
print('\n\033[1mFisher-Rao:\033[0m')
mean, std, sharpe = portfolio_tester(train_X, test_X, lambda X: l1o_cv(X, 'Fisher-Rao', np.logspace(-5,2,50)), verbose=True)

Wasserstein:
Mean: 1.0636130608724654
Std: 4.25099345470766
Sharpe: 0.2502034106156039

Kullback-Leibler:
Mean: 1.0605540167023932
Std: 4.082645013781706
Sharpe: 0.2597713034374288

Fisher-Rao:
Mean: 1.063415000284609
Std: 4.309828073764833
Sharpe: 0.24674186117954983
